# Full audio

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
num_mfcc = 0

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

coswara_feature = os.path.join(root, 'full_audio/melspectrogram_coswara_features.npy')

coughvid_feature = os.path.join(root, 'full_audio/melspectrogram_coughvid_features.npy')

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')

### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,id,age,covid_status,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,...,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,cough_path,label,error
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,healthy,0
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/AxuYWBN0jFVLINCBqIW5aZmGCdu1/cough-shallow.wav,healthy,0
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/C5eIsssb9GSkaAgIfsHMHeR6fSh1/cough-shallow.wav,healthy,0
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/YjbEAECMBIaZKyfqOvWy5DDImUb2/cough-shallow.wav,healthy,0
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/aGOvk4ji0cVqIzCs1jHnzlw2UEy2/cough-shallow.wav,healthy,0


In [ ]:
logger.info(meta_df.shape)

2022-04-30 04:45:09,629 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4465, 39)


In [ ]:
mel_feature = np.load(coswara_feature)
meta_df = pd.read_csv(meta_coswara)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
logger.info(meta_df.shape)

2022-04-30 04:45:20,322 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 39)


In [ ]:
meta_df.label.value_counts().sum()

4400

In [ ]:
mel_feature[0].shape, len(mel_feature), meta_df.shape

((224, 246), 4400, (4400, 39))

In [ ]:
DROP_COLS = [
             'cough_path',
             'error',
             'record_date',
             'id',
             'covid_status',
             'test_date'
]

meta_df.isna().sum()/4400*100 > 50

In [ ]:
meta_df.dtypes

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-30 04:37:03,253 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 33)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    3194
1    1206
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mel_feature = np.load(coswara_feature)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-30 04:37:05,141 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - 1206, 3194


In [ ]:
# get features for each class
feature_0 = mel_feature[index_0]
feature_1 = mel_feature[index_1]

In [ ]:
print(len(feature_1))

1206


In [ ]:
del mel_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:2555], feature_1[:965]), axis=0)
  val = np.concatenate((feature_0[2555:2875], feature_1[965:1086]), axis=0)
  test = np.concatenate((feature_0[2875:], feature_1[1086:]), axis=0)

  y_train = pd.concat([label0[:2555], label1[:965]], ignore_index=True)
  y_val = pd.concat([label0[2555:2875], label1[965:1086]], ignore_index=True)
  y_test = pd.concat([label0[2875:], label1[1086:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(3520, 224, 246)


In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_1d

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
X_train_1d.shape

(3520, 32, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)


Epoch 1/40
28/28 [==============================] - 27s 880ms/step - loss: 0.4375 - auc: 0.8751 - val_loss: 0.6267 - val_auc: 0.7428
Epoch 2/40
28/28 [==============================] - 24s 874ms/step - loss: 0.4046 - auc: 0.8948 - val_loss: 0.6919 - val_auc: 0.7528
Epoch 3/40
28/28 [==============================] - 24s 877ms/step - loss: 0.4146 - auc: 0.8903 - val_loss: 0.6457 - val_auc: 0.7828
Epoch 4/40
28/28 [==============================] - 25s 881ms/step - loss: 0.4024 - auc: 0.8979 - val_loss: 0.7450 - val_auc: 0.7075
Epoch 5/40
28/28 [==============================] - 25s 881ms/step - loss: 0.3792 - auc: 0.9082 - val_loss: 0.9026 - val_auc: 0.7219
Epoch 6/40
28/28 [==============================] - 25s 887ms/step - loss: 0.3506 - auc: 0.9239 - val_loss: 0.8788 - val_auc: 0.5931
Epoch 7/40
28/28 [==============================] - 25s 883ms/step - loss: 0.3238 - auc: 0.9334 - val_loss: 0.8604 - val_auc: 0.7243
Epoch 8/40
28/28 [==============================] - 25s 892ms/step - 

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

14/14 [==============================] - 1s 39ms/step - loss: 0.8309 - auc: 0.7217


[0.8309009671211243, 0.7217065095901489]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(3520, 224, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([3520, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
28/28 [==============================] - 21s 721ms/step - loss: 0.1780 - auc: 0.9782 - val_loss: 1.1276 - val_auc: 0.7342
Epoch 2/40
28/28 [==============================] - 20s 717ms/step - loss: 0.1251 - auc: 0.9871 - val_loss: 1.3336 - val_auc: 0.7391
Epoch 3/40
28/28 [==============================] - 20s 718ms/step - loss: 0.0955 - auc: 0.9927 - val_loss: 1.6918 - val_auc: 0.7410
Epoch 4/40
28/28 [==============================] - 20s 721ms/step - loss: 0.0922 - auc: 0.9913 - val_loss: 1.5559 - val_auc: 0.7313
Epoch 5/40
28/28 [==============================] - 20s 720ms/step - loss: 0.1013 - auc: 0.9898 - val_loss: 1.4363 - val_auc: 0.7272
Epoch 6/40
28/28 [==============================] - 20s 707ms/step - loss: 0.0915 - auc: 0.9921 - val_loss: 1.6955 - val_auc: 0.7306
Epoch 7/40
28/28 [==============================] - 20s 713ms/step - loss: 0.0820 - auc: 0.9929 - val_loss: 1.4879 - val_auc: 0.7651
Epoch 8/40
28/28 [==============================] - 20s 715ms/step - 

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

14/14 [==============================] - 0s 26ms/step - loss: 2.2432 - auc: 0.6833


[2.2432072162628174, 0.6833193898200989]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(3520, 224, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(5110, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(5110, 32, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([5110, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
40/40 [==============================] - 30s 718ms/step - loss: 0.0197 - auc: 0.9990 - val_loss: 2.8275 - val_auc: 0.7256
Epoch 2/40
40/40 [==============================] - 28s 694ms/step - loss: 0.0150 - auc: 0.9998 - val_loss: 2.7424 - val_auc: 0.7304
Epoch 3/40
40/40 [==============================] - 28s 703ms/step - loss: 0.0138 - auc: 0.9999 - val_loss: 2.7386 - val_auc: 0.7392
Epoch 4/40
40/40 [==============================] - 28s 694ms/step - loss: 0.0136 - auc: 0.9998 - val_loss: 2.7332 - val_auc: 0.7317
Epoch 5/40
40/40 [==============================] - 28s 696ms/step - loss: 0.0133 - auc: 0.9998 - val_loss: 2.8851 - val_auc: 0.7282
Epoch 6/40
40/40 [==============================] - 28s 698ms/step - loss: 0.0129 - auc: 0.9999 - val_loss: 3.0162 - val_auc: 0.7190
Epoch 7/40
40/40 [==============================] - 28s 695ms/step - loss: 0.0128 - auc: 0.9999 - val_loss: 3.1657 - val_auc: 0.7124
Epoch 8/40
40/40 [==============================] - 28s 702ms/step - 

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d], verbose=1)

14/14 [==============================] - 0s 26ms/step - loss: 3.7026 - auc: 0.6955


[3.702571153640747, 0.6955236792564392]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mel_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 4s 37ms/step - loss: 2.2432 - auc: 0.6833


[2.2432072162628174, 0.6833193898200989]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.9272 - auc: 0.7434


[0.9272220134735107, 0.7433698773384094]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.5131 - auc: 0.8429


[0.5131365656852722, 0.842941403388977]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.4911 - auc: 0.8453


[0.4911395311355591, 0.8453360199928284]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.5094 - auc: 0.8407


[0.5093925595283508, 0.8406504988670349]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.8309 - auc: 0.7217


[0.8309009671211243, 0.7217065095901489]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.5052 - auc: 0.8470


[0.5052400827407837, 0.8470430970191956]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 3s 28ms/step - loss: 0.4983 - auc: 0.8426


[0.4982669949531555, 0.8426169753074646]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 0.6053 - auc: 0.7707


[0.6053123474121094, 0.7707281708717346]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 28ms/step - loss: 0.5630 - auc: 0.8127


[0.5630034804344177, 0.8126721978187561]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 3.7026 - auc: 0.6955


[3.702571153640747, 0.6955236792564392]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 3.1215 - auc: 0.6651


[3.1214849948883057, 0.665148138999939]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 27ms/step - loss: 1.6447 - auc: 0.7151


[1.6446627378463745, 0.7151244878768921]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 2s 28ms/step - loss: 0.9431 - auc: 0.7679


[0.9431002140045166, 0.7678976655006409]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 3s 27ms/step - loss: 2.7260 - auc: 0.2914


[2.72597336769104, 0.2914368510246277]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
num_mfcc = 0

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coswara = os.path.join(data_root, 'cough_chunking/coswara/coswara_chunk_metadata.csv')
meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

mel_coswara = os.path.join(root, 'melspectrogram_coswara_features.npy')
mel_coughvid_1 = os.path.join(root, 'melspectrogram_coughvid_features_20k.npy')
mel_coughvid_2 = os.path.join(root, 'melspectrogram_coughvid_features_20k_40k.npy')
mel_coughvid_3 = os.path.join(root, 'melspectrogram_coughvid_features_40k_60k.npy')
mel_coughvid_4 = os.path.join(root, 'melspectrogram_coughvid_features_60k_75k.npy')
mel_coughvid_5 = os.path.join(root, 'melspectrogram_coughvid_features_9981.npy')

mfcc_coswara = os.path.join(root, f'mfcc_{num_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,uuid,age,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,is_smoker,...,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,label,sr_cough,chunk_path,num_chunk
0,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0.wav,8
1,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1.wav,8
2,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2.wav,8
3,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3.wav,8
4,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4.wav,8


In [ ]:
logger.info(meta_df.shape)

2022-04-26 02:21:41,869 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 39)


In [ ]:
DROP_COLS = [
             'sr_cough',
             'chunk_path',
             'record_date',
             'uuid',
             'test_date'
]

meta_df.isna().sum()

uuid                                 0
age                                  0
record_date                          0
is_english_proficiency               0
gender                               0
country                              0
locality                          2295
state                                0
is_returning_user                 7281
is_smoker                        17592
is_cold                          20117
is_hypertension                  21030
is_diabetes                      20985
is_cough                         19040
date_of_ct_scan                  21754
has_ctScan                       18880
ct_score                         21754
is_diarrheoa                     22514
is_fever                         20704
is_loss_of_smell                 21621
is_muscle_pain                   21229
test_type                        19560
test_date                        19560
test_status                      14906
is_using_mask                    11268
vaccination_status       

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                             0
age                              0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp                   0
is_fatigue          

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-26 02:21:42,031 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 34)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    16871
1     6007
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mel_feature = np.load(mel_coswara)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-26 02:22:00,583 - COSWARA - HYBRID MODEL CHUNKS - INFO - 6007, 16871


In [ ]:
# get features for each class
feature_0 = mel_feature[index_0]
feature_1 = mel_feature[index_1]

In [ ]:
print(len(feature_1))

6007


In [ ]:
del mel_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:13504], feature_1[:4803]), axis=0)
  val = np.concatenate((feature_0[13504:15186], feature_1[4803:5408]), axis=0)
  test = np.concatenate((feature_0[15186:], feature_1[5408:]), axis=0)

  y_train = pd.concat([label0[:13504], label1[:4803]], ignore_index=True)
  y_val = pd.concat([label0[13504:15186], label1[4803:5408]], ignore_index=True)
  y_test = pd.concat([label0[15186:], label1[5408:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(18307, 224, 236)


In [ ]:
# X_train_2d = X_train_2d[12000:13000]
# y_train_2d = y_train_2d[12000:13000]
# print(X_train_2d.shape)

In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
# X_train_1d = X_train_1d[12000:13000]
# y_train_1d = y_train_1d[12000:13000]

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Bigdl

In [ ]:
!pip install BigDL==0.8.0
clear_output()

     |████████████████████████████████| 80.5 MB 80 kB/s 
     |██████████████████████████▎     | 231.2 MB 1.7 MB/s eta 0:00:31

In [ ]:
# Install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
# Set environment variable JAVA_HOME.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

In [ ]:
from bigdl.nn.layer import *
from bigdl.util.common import *
from bigdl.optim.optimizer import *
import numpy as np
sc = SparkContext.getOrCreate(conf=create_spark_conf())
samples=[Sample.from_ndarray(np.array([1.0, 2.0]), np.array([2.0]))]
testSet = sc.parallelize(samples,1)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
X_train_1d.shape

(18307, 33, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)


Epoch 1/40
144/144 [==============================] - 91s 627ms/step - loss: 0.1325 - auc: 0.9860 - val_loss: 1.2050 - val_auc: 0.7835
Epoch 2/40
144/144 [==============================] - 90s 625ms/step - loss: 0.0423 - auc: 0.9968 - val_loss: 1.7275 - val_auc: 0.7601
Epoch 3/40
144/144 [==============================] - 90s 626ms/step - loss: 0.0310 - auc: 0.9977 - val_loss: 1.9353 - val_auc: 0.7433
Epoch 4/40
144/144 [==============================] - 89s 621ms/step - loss: 0.1473 - auc: 0.9824 - val_loss: 1.3915 - val_auc: 0.7467
Epoch 5/40
144/144 [==============================] - 90s 628ms/step - loss: 0.0370 - auc: 0.9970 - val_loss: 1.7458 - val_auc: 0.7405
Epoch 6/40
144/144 [==============================] - 91s 631ms/step - loss: 0.1133 - auc: 0.9881 - val_loss: 1.3945 - val_auc: 0.7481


In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 2s 25ms/step - loss: 1.3703 - auc: 0.7646


[1.3702754974365234, 0.764565110206604]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coswara_mel_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(18307, 224, 236)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([18307, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 93s 638ms/step - loss: 0.0871 - auc: 0.9930 - val_loss: 0.5746 - val_auc: 0.8968
Epoch 2/40
144/144 [==============================] - 91s 631ms/step - loss: 0.0434 - auc: 0.9978 - val_loss: 0.6338 - val_auc: 0.8960
Epoch 3/40
144/144 [==============================] - 91s 629ms/step - loss: 0.1049 - auc: 0.9888 - val_loss: 0.5975 - val_auc: 0.8896
Epoch 4/40
144/144 [==============================] - 91s 631ms/step - loss: 0.1482 - auc: 0.9815 - val_loss: 0.4865 - val_auc: 0.9036
Epoch 5/40
144/144 [==============================] - 91s 633ms/step - loss: 0.0861 - auc: 0.9929 - val_loss: 0.5275 - val_auc: 0.8976
Epoch 6/40
144/144 [==============================] - 91s 633ms/step - loss: 0.0721 - auc: 0.9942 - val_loss: 0.5929 - val_auc: 0.8951
Epoch 7/40
144/144 [==============================] - 91s 629ms/step - loss: 0.0737 - auc: 0.9933 - val_loss: 0.6929 - val_auc: 0.8765
Epoch 8/40
144/144 [==============================] - 9

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 2s 25ms/step - loss: 1.1358 - auc: 0.7872


[1.1357746124267578, 0.7871543169021606]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'std_coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coswara_mel_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(18307, 224, 236)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(27008, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(27008, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([27008, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
211/211 [==============================] - 137s 623ms/step - loss: 0.0396 - auc: 0.9969 - val_loss: 1.6349 - val_auc: 0.7069
Epoch 2/40
211/211 [==============================] - 131s 622ms/step - loss: 0.0209 - auc: 0.9984 - val_loss: 2.2105 - val_auc: 0.7190
Epoch 3/40
211/211 [==============================] - 132s 624ms/step - loss: 0.0253 - auc: 0.9977 - val_loss: 2.1196 - val_auc: 0.6737
Epoch 4/40
211/211 [==============================] - 130s 614ms/step - loss: 0.0238 - auc: 0.9980 - val_loss: 2.5720 - val_auc: 0.7217
Epoch 5/40
211/211 [==============================] - 132s 628ms/step - loss: 0.0263 - auc: 0.9980 - val_loss: 2.2499 - val_auc: 0.7188
Epoch 6/40
211/211 [==============================] - 132s 623ms/step - loss: 0.0242 - auc: 0.9980 - val_loss: 2.0930 - val_auc: 0.7031
Epoch 7/40
211/211 [==============================] - 131s 622ms/step - loss: 0.0246 - auc: 0.9981 - val_loss: 2.2347 - val_auc: 0.7147
Epoch 8/40
211/211 [============================

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d], verbose=1)

72/72 [==============================] - 2s 25ms/step - loss: 2.0990 - auc: 0.7158


[2.099003314971924, 0.7158037424087524]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coswara_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coswara_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coswara_mel_hybrid_model_40_128_0.001/assets


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 17s 35ms/step - loss: 1.1358 - auc: 0.7872


[1.1357746124267578, 0.7871543169021606]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.9071 - auc: 0.8137


[0.9071432948112488, 0.8136733770370483]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 30ms/step - loss: 0.6066 - auc: 0.8336


[0.6066482663154602, 0.8335580229759216]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 28ms/step - loss: 0.4818 - auc: 0.8666


[0.48182666301727295, 0.8666095733642578]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5486 - auc: 0.8011


[0.5486287474632263, 0.8011101484298706]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.3703 - auc: 0.7646


[1.3702754974365234, 0.764565110206604]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.9541 - auc: 0.7884


[0.9541028738021851, 0.7883549928665161]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5421 - auc: 0.8326


[0.5420622229576111, 0.832568347454071]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5575 - auc: 0.7900


[0.5575035214424133, 0.7900125980377197]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5113 - auc: 0.8505


[0.5113306045532227, 0.850516676902771]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coswara_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 2.0990 - auc: 0.7158


[2.099003314971924, 0.7158037424087524]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coswara_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.3288 - auc: 0.7897


[1.32882821559906, 0.7896897196769714]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coswara_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.6587 - auc: 0.8095


[0.6586807370185852, 0.8094763159751892]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coswara_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.6916 - auc: 0.8238


[0.6916368007659912, 0.8238195180892944]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coswara_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.6487 - auc: 0.7493


[0.6486642360687256, 0.7492597103118896]

## testing code

In [ ]:
from keras import Model
from keras.layers.core import Dense, Activation
from keras.layers import Conv2D, Conv1D, MaxPooling2D, Reshape, Concatenate, Dropout , MaxPooling1D, Flatten
from keras.layers import Dense, Input

in_1D = Input((7380, 128000))
# 1
model_1D = Conv1D(32, kernel_size= 5 , strides=1, activation='relu')(in_1D)
model_1D = MaxPooling1D(pool_size= 4, strides=4)(model_1D)
# 2
model_1D = Conv1D(32, kernel_size= 5 , strides=1 , activation='relu')(model_1D)
model_1D = MaxPooling1D(pool_size= 4, strides=4)(model_1D)
# 3
model_1D = Conv1D(64, kernel_size= 5 , strides=1 , activation='relu')(model_1D)
model_1D = MaxPooling1D(pool_size= 4, strides=4)(model_1D)
# 4
model_1D = Conv1D(64, kernel_size= 5 , strides=1 , activation='relu')(model_1D)
model_1D = MaxPooling1D(pool_size= 2, strides=2)(model_1D)
# 5
model_1D = Conv1D(128, kernel_size= 5 , strides= 1 , activation='relu')(model_1D)
model_1D = MaxPooling1D(pool_size= 2, strides= 2)(model_1D)
# 6
model_1D = Conv1D(128, kernel_size= 5 , strides= 1 , activation='relu')(model_1D)
model_1D = MaxPooling1D(pool_size= 2, strides= 2)(model_1D)
model_1D = Flatten()(model_1D)
model_1D = Dense(9 , activation='relu')(model_1D)

# ----------------------- 2D CNN ----------------------
in_2D = Input((7380, 128, 251))
model_2D = Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu')(in_2D)
model_2D = Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu')(model_2D)
model_2D = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(model_2D)
model_2D = Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu')(model_2D)
model_2D = Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu')(model_2D)
model_2D = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(model_2D)
model_2D = Flatten()(model_2D)
model_2D = Dense(9 , activation='relu')(model_2D)
merged = Concatenate()([model_1D, model_2D])
output = Dense(7, activation='softmax')(merged)

model_final = Model(inputs=[in_1D, in_2D], outputs=[output])

In [ ]:
model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
from tensorflow.keras.utils import plot_model
plot_model(model_final, to_file='model_final.png')

In [ ]:
model_final.summary()